In [ ]:
pip install -U langchain requests langchain_mistralai

In [ ]:
import requests
import os
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent, tool
from langchain_mistralai.chat_models import ChatMistralAI

In [ ]:
load_dotenv()
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
NVD_API_KEY = os.getenv("NVD_API_KEY")
NVD_API_URL ="https://services.nvd.nist.gov/rest/json/cves/2.0"

In [ ]:
@tool
def cve_search(cve_id: str = None, start_date: str = None, end_date: str = None, 
               keyword: str = None, cvss4_severity: str = None) -> str:
    """
    Searches for CVEs in the National Vulnerability Database.
    
    Parameters:
    - cve_id: Specific CVE ID to look up (e.g., 'CVE-2024-47195')
    - start_date: Start date in YYYY-MM-DD format (e.g., '2024-01-01')
    - end_date: End date in YYYY-MM-DD format (e.g., '2024-01-31')
    - keyword: Keyword to search for (e.g., 'Heartbleed', 'OpenSSL')
    - cvss4_severity: Filter by CVSS4 severity ('LOW', 'MEDIUM', 'HIGH', 'CRITICAL')
    
    Example inputs:
        - (cve_id='CVE-2024-47195')
        - (start_date='2024-01-01', end_date='2024-01-31')
        - (keyword='Heartbleed')
        - (cvss4_severity='HIGH') 
        - (keyword='OpenSSL', cvss4_severity='CRITICAL')
    """
    
    # Build query parameters
    params = {}
    if cve_id:
        params["cveId"] = cve_id
    if start_date and end_date:
        params["pubStartDate"] = f"{start_date}T00:00:00.000"
        params["pubEndDate"] = f"{end_date}T23:59:59.999"
    if keyword:
        params["keywordSearch"] = keyword
    if cvss4_severity:
        if cvss4_severity.upper() in {"LOW", "MEDIUM", "HIGH", "CRITICAL"}:
            params["cvssV4Severity"] = cvss4_severity.upper()
        else:
            return f"Invalid CVSS4 severity: {cvss4_severity}. Valid values are: LOW, MEDIUM, HIGH, CRITICAL"
    
    # Make API request
    headers = {"apiKey": NVD_API_KEY}
    response = requests.get(NVD_API_URL, params=params, headers=headers)
    
    if response.status_code != 200:
        return f"Error accessing NVD API: {response.status_code}"
    
    vulnerabilities = response.json().get("vulnerabilities", [])
    
    if not vulnerabilities:
        return "No vulnerabilities found matching the criteria."
    
    # Format results
    results = []
    for vuln in vulnerabilities:
        cve = vuln["cve"]
        
        # Extract basic info
        cve_id = cve["id"]
        descriptions = cve.get("descriptions", [])
        description = descriptions[0].get("value", "No description available") if descriptions else "No description available"
        published = cve.get("published", "Date unknown")
        
        # Extract CVSS metrics if available
        metrics = cve.get("metrics", {})
        cvss4_info = ""
        
        if "cvssMetricV4" in metrics:
            for metric in metrics["cvssMetricV4"]:
                base_score = metric.get("baseScore", "N/A")
                severity = metric.get("baseSeverity", "N/A")
                cvss4_info = f" [CVSS4 Score: {base_score}, Severity: {severity}]"
        
        # Format this vulnerability's entry
        results.append(f"- {cve_id} (Published: {published}){cvss4_info}: {description}")
    
    # Return formatted results
    return f"Found {len(results)} Vulnerabilities:\n" + "\n".join(results)

In [ ]:
tools = [cve_search]

llm = ChatMistralAI(
    model="mistral-tiny",
    api_key=MISTRAL_API_KEY,
    temperature=0
)

agent=initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
# Run the chatbot
while True:
    user_input = input("Ask about a vulnerability or type 'exit': ")
    if user_input == "exit":
        break
    response = agent.invoke(user_input)
    print(response['output'])